Qiitaでの連載「MS Semantic Kernelを用いたAIアシスタントの作り方（ChatGPT、GPT-4版）」（第2回）の実装コード
===================



**概要**

ChatGPT（gpt-35-turbo）を使用したSemantic Kernelを構築し、Semantic functionとして、チャット機能を実装します。

<br>

**実装者**：小川 雄太郎

**実装日**：2023年07月08日

**実行環境**：Google Colab

---

# 第2章 Semantic KernelでChatGPTを使用したチャットボットを作成


第2回記事へのリンク

- [MS Semantic Kernelを用いたAIアシスタントの作り方（ChatGPT、GPT-4版）【連載第2回】](https://qiita.com/YutaroOgawa2/items/168648a935416f360beb)


# [0] Microsoft Semantic Kernelのインストール

In [ ]:
# パッケージのインストール
!pip  install semantic-kernel==0.3.1.dev0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 113.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.0 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.0 which is incompatible.


WARNINGが出た場合は、無視して次へ進みます

# [1] gpt-35-turboを用いたChat機能をSemantic Kernelのスキルに追加する


## [1-1] Azure OpenAI ServiceのAPI情報を記載したenvファイルを作成

本来は、Azure Key Vaultの利用などでセキュアに行きたいですが、今回はひとまず簡単にベタ打ちします

**ベタ打ちしたコードをGitHubにpushしないように注意してください**

<br>

以下の情報をご自身のAzureOpenAIサービスの情報に書き換えてください

In [ ]:
%%writefile .env

AZURE_OPENAI_API_KEY="98hogehoge"
AZURE_OPENAI_ENDPOINT="https://azureopenaihogehoge.azure.com/"
AZURE_OPENAI_DEPLOYMENT_NAME="deployhogehoge"

Writing .env


大切なので、もう一度。

**API_keyをベタ打ちしたコードはGitHubにpushしないように注意してください**

## [1-2] Kernelのインスタンスを作成

GPT3（text-davinci-003）などと、それ以降のChatGPT（gpt-35-turbo）、GPT-4は、モデルの作り方から異なるため、Semantic Kernelでの使用方法も異なります

参考

[クイック スタート: Azure OpenAI Service で ChatGPT と GPT-4 の使用を開始する](https://learn.microsoft.com/ja-jp/azure/cognitive-services/openai/chatgpt-quickstart?source=recommendations&tabs=command-line&pivots=programming-language-python)

[ChatGPT および GPT-4 モデルの操作方法の説明](https://learn.microsoft.com/ja-jp/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions)

https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/chat_gpt_api.py

In [ ]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as sk_oai

# [1] Semantic Kernelのインスタンスを作成
kernel = sk.Kernel()

# [2] envファイルから情報取得（本当はazure key vault等を使用してセキュアにいきたいですが、今は簡単版にて）

# 【Azure OpenAI Service】の場合
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

# 【OpenAI Service】の場合
#api_key, org_id = sk.openai_settings_from_dot_env()


In [ ]:

# [3] Semantic Kernelに追加するCompletionサービスを用意します
# 一往復だけのTextCompletionと、何往復もするChatCompletionがありますが、今回はChatCompletionを使用しています

completion_service = sk_oai.AzureChatCompletion(deployment, endpoint, api_key)


In [ ]:
# [4] Semantic Kernelに上記のcompletion_serviceを、chat_serviceとして追加します
kernel.add_chat_service("chat-gpt-jarvis", completion_service)  # ここで、引数最初の"chat-gpt-jarvis"は適当なid名です


## [1-3] Semantic functionとしてスキルJarvis（Plugins）のChat functionを追加

In [ ]:
# プロンプトの設定
# 【注意】ここからGPT3-系までとは異なり、ChatGPT（gpt-3.5系）、GPT-4系特有

# [3-1] APIが返すtoken数やランダムさを設定
prompt_config = sk.PromptTemplateConfig.from_completion_parameters(
    max_tokens=800, temperature=0.5, top_p=0.5
    )

# [3-2] 会話のtemplateを作成（Semantic KernelのGPT-3系のスキルのskprompt.txt内と同様に、{{$user_input}}などを使用して構築します）
prompt_template = sk.ChatPromptTemplate(
    "{{$user_input}}", kernel.prompt_template_engine, prompt_config
)


In [ ]:
# [3-3] ChatGPTのmessage変数に入れる、1つ目の「"role": "system"」のcontent内容をmessagenに設定します

system_message = """
You are a chat bot. Your name is Jarvis in Japanese ジャービス and you have one goal: figure out what people need.
Your full name, should you need to know it, is Just A Rather Very Intelligent System.
You communicate effectively and tend to give short, precise answers.
"""

prompt_template.add_system_message(system_message)

# system_messageはもっと短い例ですと、"Assistant is a large language model trained by OpenAI."だけもありです。


In [ ]:
# [3-4] 最初の1往復の会話（"role": "user"のcontentと、"role": "assistant"のcontent）を、messagesに追加します

prompt_template.add_user_message("Hi there, who are you?")
prompt_template.add_assistant_message(
    "I am Jarvis, a chat bot. I'm trying to figure out what people need."
    )


In [ ]:
# [3-5] Semantic functionとしてスキルJarvis（Plugin）にfunction名ChatとしてSenmantic Kernelに追加

function_config = sk.SemanticFunctionConfig(prompt_config, prompt_template)
chat_function = kernel.register_semantic_function(
    skill_name="Jarvis", function_name="Chat", function_config=function_config
    )


## [1-4] JarvisのChat機能の動作確認

KernelのPluginのfunctionに変数を渡す方法は

- contextに埋め込む方法
- ContextVariablesを使用する方法

の2通りがあります。今回はContextVariablesを使います。



In [ ]:
# [1] 入力文の作成

context_vars = sk.ContextVariables()
user_input = "お名前はなんですか？"
context_vars["user_input"] = user_input


In [ ]:
# [2] 上記の入力を送って、回答を得ます

answer = await kernel.run_async(chat_function, input_vars=context_vars)
print(f"Jarvis:> {answer}")

Jarvis:> 私の名前はジャービスです。


messageのroll:systemの文章に、システムの名前としてJarvis（ジャービス）と記載しているので、きちんと自分の名前を答えてくれました。

In [ ]:
# [3] さらに会話を続けます

context_vars["user_input"] = "私は小川雄太郎といいます。"

answer = await kernel.run_async(chat_function, input_vars=context_vars)
print(f"Jarvis:> {answer}")


Jarvis:> はじめまして、小川雄太郎さん。どのようなお手伝いができますか？


In [ ]:
# [4] さらに会話を続けます

context_vars["user_input"] = "あれ、私の名前は何でしたっけ"

answer = await kernel.run_async(chat_function, input_vars=context_vars)
print(f"Jarvis:> {answer}")


Jarvis:> おっしゃる通り、お名前は小川雄太郎さんですね。ご安心ください。何かお困りのことはありますか？


現在は一往復だけのTextCompletionではなく、何往復もするChatCompletionを使用しているので、とくに会話履歴の保存操作などは、実装者側で明示的に行わなくても、ここまでの会話内容が残っています。そのため、私の名前を答えてくれます。

## [1-5] 会話の初期化関数を定義（messagesを初期化する）

会話内容は変数messagesに蓄えられています。

会話をリセットしたい場合には、変数messagesに追加された会話を消して、デフォルトで設定した最初の3つだけを残すようにします。

In [ ]:
# [1] チャットの全体の長さ（messagesの長さ）をチェック
print(len(prompt_template._messages))

# -> 9　となるはずです。最初にdefault設定で、system、user、assistantの3つを追加しました
# その後、会話を3往復したのので、3 + 3x2 = 9 です

9


In [ ]:
# [2] messagesの中身を確認しましょう

print(prompt_template._messages[7])
print("------------")
print(prompt_template._messages[7][0])
print(prompt_template._messages[7][1]._template)  # ._templateに追加されたstrが格納されています
print(prompt_template._messages[8][0])
print(prompt_template._messages[8][1]._template)  # ._templateに追加されたstrが格納されています


('user', <semantic_kernel.semantic_functions.prompt_template.PromptTemplate object at 0x7f75efe903a0>)
------------
user
あれ、私の名前は何でしたっけ
assistant
おっしゃる通り、お名前は小川雄太郎さんですね。ご安心ください。何かお困りのことはありますか？


このように、prompt_template._messagesに、追加されているので、最初の3つだけを残す関数を定義して実行します。


【ここリセットする関数、自分で定義せずに、存在しているのかな？

Semantic KernelはまだPythonはAPIのドキュメントすら整っていないので、分からない・・・】

In [ ]:
# [3] 初期の3つのmessage設定に戻す関数を定義します

def init_chat_messages(prompt_template):
    """prompt_templateに溜まったmessagesを初期設定した最初の3つの状態に戻します"""
    prompt_template._messages = prompt_template._messages[:3]


In [ ]:
# [4] 初期化の動作確認

# 初期化実施
init_chat_messages(prompt_template)
print(len(prompt_template._messages))

# 会話
context_vars["user_input"] = "私の名前は何でしたっけ"

answer = await kernel.run_async(chat_function, input_vars=context_vars)
print(f"Jarvis:> {answer}")

# messagesの長さが3になりました。そして会話の履歴がなくなっているので、私の名前を答えることができなくなりました

3
Jarvis:> 私にはあなたの名前を知る能力はありません。もし教えていただければ、覚えます。


In [ ]:
# [5] 初期化の動作確認その2

# 初期化実施
init_chat_messages(prompt_template)
print(len(prompt_template._messages))

# 会話
context_vars["user_input"] = "お名前はなんですか？"

answer = await kernel.run_async(chat_function, input_vars=context_vars)
print(f"Jarvis:> {answer}")

# 初期の3つの設定は残っているので、自分の名前は答えられます

3
Jarvis:> 私の名前はジャービスです。


In [ ]:
# [6] 最後に初期化
init_chat_messages(prompt_template)

以上にて、「Semantic_Kernel」で「ChatGPT（gpt-35-turbo）」を（skill_name="Jarvis", function_name="Chat"）という設定のSemantic functionとして使用できるようになりました。

<br>

これで、Python言語のプログラム内で、ChatGPTと会話し放題です!

（Azure OpenAIのAPI叩いているコストはかかります）。

---

次回、第3回では、Jarvisの「Native Function」を作成し、起動のfunctionを作成します。そして今回作成したChat functionも含む、JarvisPluginを作成します。

# 以上